# 06 난수

### 6.1 유사 난수
* 컴퓨터는 이론적으로 완벽한 난수 생성이 불가
* 따라서 컴퓨터는 난수표, 난수 알고리즘, 알고리즘 초기화에 사용할 seed값으로 난수를 만듦. 이러한 것을 유사 난수 pseudo random이라 함.
* 알고리즘 대신 열 잡음, 광전자 등 신호의 노이즈를 이용하여 시드가 필요 없는 하드웨어 랜덤 번호 생성기hardware random number generator (HRNG)를 사용하기도 함.
* 유사 난수의 특징은 예측할 수 없는 난수를 생성한다는 것.

In [1]:
import random

for i in range(1, 10):
    # seed 값을 현재 시간(타임스탬프)로 설정합니다.
    random.seed()
    print(random.randrange(1, 10))

9
4
3
4
5
4
5
3
3


* 유사 난수 알고리즘에서 가장 대중적인 것은 메르센 트위스터 Mersenne Twister이다.
* 메르센 트위스터는 충분히 큰 난수 주기와 균일한 난수 분포 그리고 빠른 생성 속도가 장점이다.

### 시드 값의 중요성
* 유사 난수를 사용할 때는 시드 값이 적절하게 적용되야 한다.
* 메르센 트위스터를 포함한 모든 유사 난수 알고리즘은 시드값이 같으면 같은 순서의 난수를 얻기 때문.

In [2]:
import random

for i in range(1, 10):
    # seed 값을 0으로 설정
    random.seed(0)
    print(random.randrange(1, 10))


7
7
7
7
7
7
7
7
7


* 624개의 난수만 확보하면 다음 난수를 예측할 수 있다.
* 타임스탬프처럼 단순하게 증가하는 난수도 예측이 쉽다.
* 예측 가능한 난수는 암호학적으로 안전하지 않다. 아래의 경우엔 예측 가능한 난수를 사용하면 안되는 경우들이다.
* 엑세스 토큰, API 시크릿, OTP 인증 코드, 접근 URL, 1회용 암호 등 권한 증명에 사용하는 값 생성
* 생성된 난수를 기준으로 게임 아이템/재화를 결정할 때 (랜덤 박스)

### 6.2 암호학적으로 안전한 난수 secure random
* 생성 속도가 느리지만 시드 값을 사용하지 않아 예측이 불가하다.
* 리눅스/유닉스는 /dev/urandom 파일을 읽은 값을, 윈도우는 BCryptGenRandom() 함수에서 반환 값을 사용하여 노이즈에 기반한 난수를 얻을 수 있다. 파이썬은 OS와 상관없이 암호학적으로 안전한 난수를 사용할 수 있다.

In [3]:
import os
import struct

for i in range(1, 10):
    # 운영체에서 제공하는 기능을 사용해 랜덤하게 생성된 4바이트 값을 읽습니다.
    random_four_byte = os.urandom(4)
    # 4바이트 값을 정수로 변환한 후, 출력합니다.
    random_integer = struct.unpack("<L", random_four_byte)[0]
    print('hex=' + random_four_byte.hex() + ", integer=" + str(random_integer))

hex=c704769f, integer=2675311815
hex=9919f857, integer=1475877273
hex=80dc8ddb, integer=3683507328
hex=1fd01081, integer=2165362719
hex=4ecf1cc1, integer=3239890766
hex=869d4ffb, integer=4216298886
hex=6bbee3a8, integer=2833497707
hex=3ecae769, integer=1776798270
hex=18405904, integer=72957976


### 난수 생성 속도 비교
* random.randrange() 함수는 암호학적으로 안전한 난수 생성보다 더 느린 결과가 나온다. randrange()는 파이썬 인터프리터를 통해 동작하고, 다른 함수 (random과 urandom)는 C 코드로 작성된 모듈만 호출하여 더 빠르기 때문이다.

In [4]:
#!/usr/bin/python3

import random
import time
import os

# 유사 난수 값을 백만 번 생성한 후, 성능을 측정합니다.
random.seed()
prng_t1 = time.monotonic()
for i in range(1, 1000000):
    random.random()

prng_t2 = time.monotonic()

# 암호학적으로 안전한 난수 값을 백만 번 생성한 후, 성능을 측정합니다.
srng_t1 = time.monotonic()
for i in range(1, 1000000):
    random_four_byte = os.urandom(4)

srng_t2 = time.monotonic()

print("Elapsed time(PRNG)={0}".format(prng_t2-prng_t1))
print("Elapsed time(SRNG)={0}".format(srng_t2-srng_t1))

Elapsed time(PRNG)=0.07800000000861473
Elapsed time(SRNG)=0.21799999999348074


### 6.3 공정한 난수, 셔플 백 shuffle bag
* 난수를 제어하는 기법으로 발생할 수 있는 모든 가능성을 한 가방에 넣고 섞는 방법을 말한다.

In [5]:
import random

# 당첨 확률: 30%
WIN_RATE = 0.3
# 뽑기 횟수: 10개
NUMBER_OF_DRAWS = 10

# 뽑기 컨테이너와 승/패 개수
draws = []
win_draws = int(NUMBER_OF_DRAWS * WIN_RATE)
loss_draws = NUMBER_OF_DRAWS - win_draws
print("win={0} / loss={1}".format(win_draws, loss_draws))

# 당첨 제비를 넣습니다.
for i in range(0, win_draws):
    draws.append(1)

# 그 다음 꽝 제비를 넣습니다.
for i in range(0, loss_draws):
    draws.append(0)

# 제비를 섞습니다.
random.seed()
random.shuffle(draws)

# 제비 출력
print(draws)

win=3 / loss=7
[0, 0, 1, 1, 0, 0, 0, 0, 1, 0]


* 전체 요소가 너무 많거나, 희박한 경우, 모든 경우의 수를 담기 위해 컨테이너의 크기가 커진다는 단점이 있다.
* 예를 들어, 희귀한 게임 아이템은 0.01%의 확률을 갖는데, 이를 셔플백으로 구현하기란 매우 어렵다. 사용자별로 셔플 백을 만들거나, 모든 사용자가 셔플 백 컨테이너 1개를 함께 사용하는 것이 현실적으로 불가하기 때문이다.

### 6.4 난수를 사용하는 목적
* 식별자 생성 : 요청, 작업 ID와 같은 식별자 생성은 겹치지 않는 수를 빠르게 만드는 것이 중요하다.
* OTP 또는 액세스 토큰 발급 : 요청, 작업 ID를 만들 때와 같이 서로 다른 서버에서 생성한 두 값이 충돌하지 않게 해야 하며 반드시 암호학적으로 안전한 난수여야 한다.
* 게임의 규칙으로 사용할 경우 : 랜덤 박스 / 전투 규칙 등에 사용한다.